# Tensorflow basics

In deze notebook worden een aantal basisconcepten van tensorflow toegelicht en bekeken aan de hand van een aantal voorbeelden.

## Tensors

Het basisconcept van tensorflow is een tensor.
Dit is een multi-dimensionele rij met een uniform datatype of dtype.
Dit concept komt heel sterk overeen met een array van Numpy.
Alle tensors zijn onwijzigbaar of immutable, net zoals spark dataframes.
Dit houdt in dat hun waarden niet kunnen gewijzigd worden.
Alle operaties op een tensor maken dus een nieuwe tensor aan.

Een tensor aanmaken kan door middel van de constant functie.

Op tensors kunnen de volgende operaties uitgevoerd worden
* add
* multiply
* matrix multiplication
* en andere wiskunde operaties zoals softmax, maximum vinden, ... (zie hiervoor ook de [math package](https://www.tensorflow.org/api_docs/python/tf/math))

In [1]:
#add

#multiply

#matrix multiplication

#find maximum

#find index of maximum

#compute softmax for each element in tensor

**Shapes**

De dimensies van een tensor zijn zeer belangrijk.
Bij communicatie over de tensors worden de volgende termen gebruikt:
* Axis: Dit is een specifieke dimensie van de tensor
* Rank: Het aantal dimensies waarover de tensor beschikt
    * Eenvoudig getal: 0
    * Een enkelvoudige rij: 1
    * Een matrix: 2
* Shape: De lengte van elk van de dimensies van de tensor. **Let op**: Buiten een aantal uitzondering moet een tensor steeds rechthoekig zijn. Dit wil zeggen dat elk element op een axis dezelfde lengte moet hebben.
* Size: Het totaal aantal elementen in de tensor. Dit is het product van de lengtes van elke dimensie

Deze waarden van een tensor kunnen als volgt berekend worden:

**Indexing:**

Tensorflow maakt gebruik van de standaard python indexing regels:
* Indexes beginnen op 0
* Negatieve indexes tellen vanaf het einde
* Dubbelpunt wordt gebruikt om slices of bereiken te kiezen volgens start:stop:end

**Manipuleren van een tensor**

Reshaping of het aanpassen van de shape van een tensor kan vaak heel handig zijn.
Hiervoor kan je de reshape functie gebruiken. 
Deze operatie is heel rekenefficient omdat de data ongewijzigd blijft.

Voor ML moet er vaak een matrix van features (figuren) omgezet worden naar een 1-dimensionale vector/rij/tensor om het te kunnen aanbieden aan een neuraal netwerk. Deze operatie wordt ook flatten genoemd.

Typisch is het enkel aangeraden om een reshape te doen om dimensies met een lengte van 1 weg te laten of naburige axis te combineren of te splitsen. Indien je meer willekeurige zaken doet gaan de resultaten niet altijd bruikbaar zijn omdat de volgorde van dimensies niet gerespecteerd blijft.

Daarnaast zijn er nog een aantal interessante kenmerken van tensors die zelf verder bestudeerd kunnen worden.
Enkele voorbeelden hiervan zijn:
* Broadcasting
* Data types
* Sparse tensors
* Ragged tensors
* String tensors

**Oefening:**

Maak twee tensors aan met respectievelijk een shape van 3x4 en 4x3.
* Bereken het product van beide matrices. Kan dit product in beide richtingen berekend worden?
* Wat is het gemiddelde van de eerste tensor?
* Wat is het minimum van de tweede tensor?
* Bereken het negatieve van de eerste tensor. Dit betekend dat elk element vermenigvuldigd wordt met -1.

## Variables

Met tensors kan je reeds heel wat berekeningen uitvoeren.
Echter is het onmogleijk om een persistente state te hebben die gemanipuleerd wordt door het programma omdat ze immutable zijn.
Deze persistente state kan bijvoorbeeld een tensor van de gewichten zijn in het model of andere parameters die gewijzigd of getrained moeten worden.

Om deze state bij te houden moet er gebruikt gemaakt worden van een Variable.
Deze kan aangemaakt worden door gebruik te maken van de klasse tf.Variable().
Een object van deze klasse stelt een tensor voor dat wel kan aangepast worden door er operaties op uit te voeren.

Zoals hierboven aangehaald moeten er specifieke operaties gebruikt worden om de waarde van de tensor aan te passen.
Een aantal mogelijke operaties hiervoor zijn:
* tf.Variable.assign()
* tf.Variable.assign_add()
* tf.Variable.assign_sub()
* Andere functies kan je [hier](https://www.tensorflow.org/api_docs/python/tf/Variable) vinden.

Let op dat hierbij de aangemaakt tensor herbruikt wordt. Hierdoor is het dus niet mogelijk om een andere size in te stellen voor de variable.

In de code hieronder zie je een aantal code voorbeelden voor te werken met variabelen:

Voor performantie probeert Tensorflow steeds de tensors en variabelen op het snelste toestel te plaatsen dat compatibel is met zijn datatype. 
Dit betekend dat de meeste variabelen op de **GPU** bewaard worden indien er 1 beschikbaar is.
Dit gedrag kan aangepast worden doro middel van de tf.device functie.
Indien je hiervan gebruik maakt kan je er ook voor zorgen dat de locatie ingesteld wordt op 1 toestel en de uitvoering op een ander gedaan wordt.
Dit introduceert wel een delay omdat de data gekopieerd moet worden naar het andere toestel.
Dit wordt soms gedaan als je meerdere GPU's gebruikt maar slechts 1 kopie wilt van de variabelen.

## Eager vs graph execution

In de code hierboven is steeds gebruik gemaakt van zogenoemde **eager execution**. 
Dit houdt in dat de code uitgevoerd wordt op dezelfde manier dat de python code uitgevoerd wordt, namelijk:
* Lijn per lijn wordt de code geinterpreerd
* Operatie na operatie wordt uitgevoerd
* De resultaten worden na elke operatie teruggegeven aan de python interpreter

Ook al heeft deze eager execution een aantal voorbeelden (met name dat het vlotter programmeert in python) zijn er ook een aantal problemen.
Het grootste probleem is dat je code enkel kan uitgevoerd worden op een systeem dat python kan draaien.
Daarnaast is het ook niet het meest performante systeem omdat de interpreter van python geen code kan optimaliseren doordat het lijn per lijn werkt.
Een alternatieve manier van uitvoering dat beschikbaar gesteld wordt door tensorflow wordt **graph execution** genoemd.
Dit houdt in dat je in python code een graaf opsteld van de operaties die voor een bepaalde input moeten uitgevoerd worden.
Tensorflow kan dan alle operaties in de volledige graaf in 1 keer en in parallel uitvoeren.
Meer informatie over deze manier van code uitvoeren kan je [hier](https://www.tensorflow.org/guide/intro_to_graphs) vinden.

Een eerste belangrijke opmerking is dat een graph gemaakt wordt voor elke unieke set van inputs (dus op basis de specifieke waarden).
Dit wordt ook wel de handtekening of (input) signature genoemd van de graph.
Dit is belangrijk omdat op deze manier de resultaten gecached kunnen worden en de optimalisatie niet steeds opnieuw moet gebeuren.

De omzetting van eager execution naar graph execution gaat vrij automatisch en probleemloos.
De meeste standaard python code wordt zonder problemen en volledig omgezet.
Een eerste belangrijke opmerking is hoe de graph execution werkt.
Bekijk hiervoor de output van onderstaande voorbeeld.

Het valt op dat als je bovenstaande code uitvoert als graph execution, dat de output slechts 1 keer uitgeprint wordt. 
Dit komt omdat de graph geoptimaliseerd wordt door het "tracing" proces.
De print wordt hierdoor niet meegenomen omdat het geen deel uitmaakt van de tensorflow operaties en ook geen impact heeft.
De print functie wordt ook wel een python side effect genoemd.
Als je toch steeds een print wil uitvoeren moet je gebruik maken van de functie **tf.print()**.

Wanneer de graph execution uitgeschakeld wordt door middel van de run_functions_eagerly functie  dan zien we dat de print wel drie keer uitgevoerd wordt.
Typisch worden bij graph executions enkel de volgende functies uitgevoerd:
* De return value
* Functies met well-known side effects zoals
    * tf.print
    * tf.debugging (asserts bvb)
    * aanpassingen van tf.Variable objecten
    
Dit type gedrag wordt ook non-strict execution genoemd omdat niet alle stappen van het programma doorlopen worden.
Dit heeft ook als gevolg dat het kan zijn dat exceptions niet opgeworpen worden als het gebeurt in een functie die niet opgeroepen wordt.
**Reken dus niet op het opwerpen van een exception**.
Een voorbeeld hiervan kan bestudeerd worden in onderstaande voorbeeld.

**Performantie boost**

Gebruik maken van graph execution heeft typisch een performantie winst tot gevolg, afhankelijk van het type van berekening dat je uitvoert.
Hieronder staat een voorbeeld waarbij een aantal matrixberekeningen uitgevoerd worden en de uitvoeringstijd voor zowel eager als graph execution wordt berekend.

Let wel op dat deze performantie boost enkel ervaren wordt wanneer een bepaalde functie meermaals uitgevoerd wordt.
De eerste keer dat de functie uitgevoerd wordt is er extra rekentijd nodig om de graph op te bouwen en te optimaliseren (**tracing** genoemd).
Deze extra kost op vlak van rekentijd wordt echter snel terugverdiend door de volgende functies.

**Best practices voor graph execution**

* **Wissel tussen eager en graph execution** om vast te stellen op welk punt de modes wijzigen.
* Maak tf.Variables aan buiten de python functie en wijzig ze binnen de functie. Dit geldt ook voor andere objecten zoals layers, Models, optimizers, ...
* Vermijd functies te schrijven die afhangen van standaard python variabelen behalve tf.Variables en Keras objecten.
* Geef de voorkeur aan tensors en en tensorflow types als inputs van functies
* Plaats zoveel mogelijk bewerkingen in tf.functions om de performantiewinst te optimaliseren.